### Creating tasks

The event loop only keeps weak references to tasks. 

A task that isn’t referenced elsewhere may get garbage collected at any time, even before it’s done. For reliable “fire-and-forget” background tasks, gather them in a collection (A)

In [ ]:
import asyncio


async def some_coro(param):
    await asyncio.sleep(1)
    print(f"Completed coro with param: {param}")


background_tasks = set()  # A 1

for i in range(10):
    task = asyncio.create_task(some_coro(param=i))

    # Add task to the set. This creates a strong reference.
    background_tasks.add(task)  # A 2

    # To prevent keeping references to finished tasks forever,
    # make each task remove its own reference from the set after
    # completion:
    task.add_done_callback(background_tasks.discard)  # A 3

await asyncio.sleep(0)

Completed coro with param: 0
Completed coro with param: 1
Completed coro with param: 2
Completed coro with param: 3
Completed coro with param: 4
Completed coro with param: 5
Completed coro with param: 6
Completed coro with param: 7
Completed coro with param: 8
Completed coro with param: 9


### Task cancellation

- To cancel a task, call its `cancel()` method. (A)
- This schedules a `CancelledError` to be thrown into the wrapped coroutine on its next opportunity to run. (B)

In [ ]:
import asyncio


async def some_coro(param):
    try:
        await asyncio.sleep(1)
    except asyncio.CancelledError:  # B
        print(f"Cancelled coro with param: {param}")
        raise
    print(f"Completed coro with param: {param}")


background_tasks = set()

for i in range(10):
    task = asyncio.create_task(some_coro(param=i))
    background_tasks.add(task)
    task.add_done_callback(background_tasks.discard)

await asyncio.sleep(0)
background_tasks.pop().cancel()  # A
background_tasks.pop().cancel()
background_tasks.pop().cancel()

True

Cancelled coro with param: 6
Cancelled coro with param: 9
Cancelled coro with param: 5


Completed coro with param: 0
Completed coro with param: 1
Completed coro with param: 2
Completed coro with param: 3
Completed coro with param: 4
Completed coro with param: 7
Completed coro with param: 8


### Task groups

Async context manager (A) that can create task and gives reliable way to wait for all tasks to finish.

- Cancellation
    - Cancelling a task do not affect the rest. (D)
    - Cancelling all tasks can be done by having a task raising a ignored exception (B).
- Exception handelling: 
    - If a task raises an exception, TaskGroup will cancel the remaining scheduled tasks (C).

In [12]:
import asyncio


async def some_coro(param):
    await asyncio.sleep(1)
    print(f"Completed coro with param: {param}")


async with asyncio.TaskGroup() as tg:  # A 1
    for i in range(3):
        tg.create_task(some_coro(param=i))  # A 2

Completed coro with param: 0
Completed coro with param: 1
Completed coro with param: 2


In [13]:
import asyncio


async def some_coro(param):
    await asyncio.sleep(1)
    print(f"Completed coro with param: {param}")


async with asyncio.TaskGroup() as tg:
    tasks = []
    for i in range(3):
        tasks.append(tg.create_task(some_coro(param=i)))
    tasks[0].cancel()  # D

Completed coro with param: 1
Completed coro with param: 2


In [18]:
import asyncio


class TerminateTaskGroup(Exception):
    pass


async def force_terminate_task_group():
    raise TerminateTaskGroup()


async def job(task_id, sleep_time):
    print(f"Task {task_id}: start")
    await asyncio.sleep(sleep_time)
    print(f"Task {task_id}: done")


async def main():
    try:
        async with asyncio.TaskGroup() as group:
            group.create_task(job(1, 0.5))
            group.create_task(job(2, 1.5))
            await asyncio.sleep(1)
            group.create_task(force_terminate_task_group())
    except* TerminateTaskGroup:  # exception group instead of normal exception B
        print("Task group terminated due to TerminateTaskGroup exception.")


await main()

Task 1: start
Task 2: start
Task 1: done
Task group terminated due to TerminateTaskGroup exception.


In [21]:
async def main():
    async with asyncio.TaskGroup() as group:
        group.create_task(job(1, 0.5))
        group.create_task(job(2, 1.5))
        await asyncio.sleep(1)
        group.create_task(force_terminate_task_group())  # C
        # raise Exception


await main()

Task 1: start
Task 2: start
Task 1: done


  + Exception Group Traceback (most recent call last):
  |   File "C:\Users\yzdom\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\yzdom\AppData\Local\Temp\ipykernel_10924\443588019.py", line 9, in <module>
  |     await main()
  |   File "C:\Users\yzdom\AppData\Local\Temp\ipykernel_10924\443588019.py", line 2, in main
  |     async with asyncio.TaskGroup() as group:
  |                ~~~~~~~~~~~~~~~~~^^
  |   File "c:\Users\yzdom\AppData\Local\Programs\Python\Python313\Lib\asyncio\taskgroups.py", line 71, in __aexit__
  |     return await self._aexit(et, exc)
  |            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\yzdom\AppData\Local\Programs\Python\Python313\Lib\asyncio\taskgroups.py", line 173, in _aexit
  |     raise BaseExceptionGroup(
  |     ...<2 lines>...
  |     ) from None
  | ExceptionGroup: unhandled errors in a TaskGroup (1 s

### asyncio.gather

Used to run coros concurrently. Coroutine are automatically scheduled as tasks (A).

#### Cancellation
- Cancel the gather() to cancel all submitted tasks (B).
- Cancelling a task raises CancelledError in the gather (C).


In [35]:
gather_fut = asyncio.gather(asyncio.sleep(1))
print(type(gather_fut))

<class 'asyncio.tasks._GatheringFuture'>


In [ ]:
async def cancel_coro(coro):
    coro.cancel()
    print("awaitable cancelled.")

In [27]:
import asyncio


async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f


async def main():
    result_lst = await asyncio.gather(
        factorial("A", 2),  # A
        asyncio.create_task(factorial("B", 3)),
    )
    print(f"{result_lst = }")


await main()

Task B: Compute factorial(3), currently i=2...
Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=3...
Task A: factorial(2) = 2
Task B: factorial(3) = 6
result_lst = [2, 6]


In [49]:
async def main():
    gfut = asyncio.gather(
        factorial("A", 2),
        asyncio.create_task(factorial("B", 3)),
    )
    try:
        await asyncio.gather(
            gfut,
            cancel_coro(gfut),  # B
        )
    except asyncio.CancelledError:
        print("Main caught gather future cancellation.")


await main()

Task B: Compute factorial(3), currently i=2...
Task A: Compute factorial(2), currently i=2...
awaitable cancelled.
Main caught gather future cancellation.


In [50]:
async def main():
    task = asyncio.create_task(factorial("B", 3))
    try:
        result_lst = await asyncio.gather(
            factorial("A", 2),
            task,
            cancel_coro(task),  # C
        )
        print(f"{result_lst = }")
    except asyncio.CancelledError:
        print("Main caught awaitable cancellation.")


await main()

Task B: Compute factorial(3), currently i=2...
Task A: Compute factorial(2), currently i=2...
awaitable cancelled.
Main caught awaitable cancellation.


Task A: factorial(2) = 2


#### Exception handling
- return_exceptions: exception in result. (D)
- not return_exceptions: propagate the first exception and continue the rest. (E)
    - Also, when gather is marked done due to an exception, cancelling the gather will have no effect on the submitted tasks. (F)

In [53]:
async def main():
    task = asyncio.create_task(factorial("B", 3))
    try:
        gather = asyncio.gather(
            factorial("A", 2),
            task,
            cancel_coro(task),
            return_exceptions=True,  # D
        )
        result_lst = await gather
        print(f"{result_lst = }")
    except asyncio.CancelledError:
        print("Main caught awaitable cancellation.")


await main()

Task B: Compute factorial(3), currently i=2...
Task A: Compute factorial(2), currently i=2...
awaitable cancelled.
Task A: factorial(2) = 2
result_lst = [2, CancelledError(''), None]


In [56]:
async def main():
    task = asyncio.create_task(factorial("B", 3))
    try:
        gather = asyncio.gather(
            factorial("A", 2),
            task,
            cancel_coro(task),  # E
        )
        result_lst = await gather
        print(f"{result_lst = }")
    except asyncio.CancelledError:
        print("Main caught awaitable cancellation.")
        gather.cancel()  # F
        print("Gather cancelled from except block.")


await main()

Task B: Compute factorial(3), currently i=2...
Task A: Compute factorial(2), currently i=2...
awaitable cancelled.
Main caught awaitable cancellation.
Gather cancelled from except block.


Task A: factorial(2) = 2
